In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
import numpy as np
from sys import stdout

/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


## Procesando el archivo pdb
Cargamos el pdb del sistema y arreglamos con PDBFixer los posibles defectos como carencia de residuos, terminales o átomos:

In [3]:
system_modeller = m3t.fix_pdb_structure('MisL_Phyre.pdb',output_form='openmm.Modeller')

In [4]:
system_modeller_minimized = m3t.energy_minimization(system_modeller)

Potential Energy before minimization: 1942911897791091.2 kJ/mol
Potential Energy after minimization: -24168.1397641843 kJ/mol


In [5]:
m3t.view([system_modeller, system_modeller_minimized])

NGLWidget()

In [6]:
initial_positions = m3t.get(system_modeller_minimized, coordinates=True)
topology = m3t.convert(system_modeller_minimized,'openmm.Topology')

In [7]:
forcefield = app.ForceField('amber99sbildn.xml')

In [8]:
system = forcefield.createSystem(topology, constraints=app.HBonds)

## Creando la simulación

In [9]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 0*unit.kelvin
pressure    = None

In [10]:
friction   = 100000.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = mm.LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

In [11]:
simulation_time = 0.005*unit.nanoseconds
saving_time = 0.0005*unit.nanoseconds
simulation_steps = round(simulation_time/step_size)
saving_steps     = round(saving_time/step_size)
num_steps_saved  = round(simulation_steps/saving_steps)

In [12]:
print('Simulation steps:', simulation_steps)
print('Saving steps:', saving_steps)
print('Saved steps:', num_steps_saved)

Simulation steps: 2500
Saving steps: 250
Saved steps: 10


In [13]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [14]:
simulation = app.Simulation(topology, system, integrator, platform, properties)

In [15]:
simulation.context.setPositions(initial_positions)
simulation.context.setVelocitiesToTemperature(0*unit.kelvin)

In [16]:
simulation.reporters.append(app.DCDReporter('trajectory.dcd', saving_steps))
simulation.reporters.append(app.StateDataReporter(stdout, saving_steps, step=True,
                            progress=True, remainingTime=True, speed=True,
                            totalSteps=simulation_steps, separator='\t'))

In [17]:
simulation.step(simulation_steps)

#"Progress (%)"	"Step"	"Speed (ns/day)"	"Time Remaining"
10.0%	250	0	--
20.0%	500	77.5	0:04
30.0%	750	77	0:03
40.0%	1000	76.8	0:03
50.0%	1250	76.9	0:02
60.0%	1500	76.9	0:02
70.0%	1750	76.8	0:01
80.0%	2000	76.9	0:01
90.0%	2250	76.7	0:00
100.0%	2500	74.8	0:00


In [18]:
traj_md = m3t.load('trajectory.dcd','mdtraj',topology=topology)

In [19]:
m3t.view(traj_md)

NGLWidget(count=10)